### Originally tried to run on AWS. Was trying to test different tf-idf things. I'll now opt instead to hone in on # max_features first, then maybe get back to grid searching...

In [1]:
import cPickle as pickle
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.pipeline import Pipeline
from scipy import interp
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pickle.load(open('labeledhate_5cats.p', 'rb'))

In [ ]:
X = df.body
y = df.label

# Binarize the output
y = label_binarize(y, classes=['NotHate', 'SizeHate', 'GenderHate', 'RaceHate', 'ReligionHate'])
n_classes = y.shape[1]
classes=['NotHate', 'SizeHate', 'GenderHate', 'RaceHate', 'ReligionHate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
stemmer = SnowballStemmer("english")

def stem_tokens(tokens, stemmer):
    stemmed=[]
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems


In [ ]:
text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', tokenizer=tokenize, decode_error='ignore')),
                     ('clf', OneVsRestClassifier(MultinomialNB())),
    ])

text_clf = text_clf.fit(X_train, y_train)
# pred = text_clf.predict(X_test)
# text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', tokenizer = tokenize, decode_error='ignore')),


In [ ]:
from sklearn.grid_search import GridSearchCV
parameters = {'tfidf__use_idf': (True, False),
              'tfidf__max_features': (5000, 2000, 500, 10000),
              'tfidf__ngram_range': [(1, 1), (1,2)],
             }

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)


In [ ]:
gs_clf = gs_clf.fit(X_train, y_train)

In [2]:
print("The best parameters are %s with a score of %0.2f"
      % (gs_clf.best_params_, gs_clf.best_score_))

NameError: name 'gs_clf' is not defined